In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, f1_score, cohen_kappa_score
from sklearn.utils import resample

In [2]:
### Read in the data

numerical = pd.read_csv('./files_for_lab/numerical.csv')
categorical = pd.read_csv('./files_for_lab/categorical.csv')
targets = pd.read_csv('./files_for_lab/target.csv')

In [3]:
print(numerical.shape)
print(categorical.shape)
print(targets.shape)

(95412, 315)
(95412, 22)
(95412, 2)


In [4]:
targets["TARGET_B"].value_counts(dropna = False)

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [5]:
### I will predict target b, so I will drop target d

targets.drop(["TARGET_D"], axis = 1)

,TARGET_B
0,0
1,0
2,0
3,0
4,0
...,...
95407,0
95408,0
95409,0
95410,1


In [6]:
### here I am enconding the categorical data

encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
encoded_categorical.shape

(95412, 319)

In [7]:
data = pd.concat([numerical, encoded_categorical, targets], axis = 1)
data.dropna(axis= 0,inplace=True)
display(data.head())
data.shape

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

(95412, 636)

In [8]:
### x, y split

X = data.drop(["TARGET_B"], axis = 1)
y = data["TARGET_B"]

In [9]:
### train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
### generating train set again to upsample the donor samples

trainset = pd.concat([X_train, y_train], axis=1)
trainset.shape

(76329, 636)

In [11]:
### creating donors and not_donors dataset for upsampling

donors = trainset[trainset["TARGET_B"] == 1]
not_donors = trainset[trainset["TARGET_B"] == 0]
print(not_donors.shape)
print(donors.shape)

(72486, 636)
(3843, 636)


In [12]:
### upsampling donors dataset

donors_upsample = resample(donors,
             replace=True,
             n_samples=len(not_donors),
             random_state=42)

print(donors_upsample.shape)
display(donors_upsample.head())

(72486, 636)


,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,TARGET_D,TARGET_B
65471,0,49.000000,4,0,9,0,0,0,0,2,23,0,0,1198,295,340,99,0,0,41,59,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,32,35,21,34,38,51,39,41,20,22,35,26,10,4,2,1,15,17,32,20,16,5,2,4,13,18,69,47,24,13,24,22,4,50,287,346,48,46,0,50,50,44,0,0,0,426,481,1,1,41,59,97,3,0,77,87,22,19,94,6,58,2,56,5,15,0,0,0,0,43,55,2,2,2,2,2,15,1,3,20,46,48,20,7,46,4,0,8,7,44,0,0,2,9,23,3560.0,373.0,718.0,139,130,169,156,4437,58,24,9,9,0,0,0,0,0,61,24,9,6,0,0,0,0,0,8,6,0,40,28,72,0,75,12,8,6,0,2,0,5,0,0,12,18,19,2,71,67,61,70,50,61,65,18,77,0,19,8,8,6,13,16,9,0,19,0,7,11,2,0,0,0,2,10,0,2,2,20,14,2,17,0,17,5,5,2,2,23,0,0,74,0,0,123,5,25,18,26,17,8,0,2,49,2,41,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,94,97,3,0,0,78,49,0,6,14,0,19,65,88,99,0,0,0,76,5,20,0,0,0,95,5,99,99,96,5,0,2,0,17,42,6,13,95.0,6,2,10.0,25.0,25.0,4,15.833333,55358,1,2,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [13]:
### here I am concatenating both datasets again and doing the x, y split for the train set again

trainset_new = pd.concat([donors_upsample, not_donors], axis = 0)
trainset_new = trainset_new.sample(frac =1) 
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']

In [14]:
print(X_train.shape)

(144972, 635)


In [15]:
### this is the random forest model

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8)
clf.fit(X_train, y_train)

print("The R2 for the Random Forest in the TRAIN set is {:.2f}".format(clf.score(X_train, y_train)))
print("The R2 for the Random Forest in the TEST  set is {:.2f}".format(clf.score(X_test, y_test)))


The R2 for the Random Forest in the TRAIN set is 0.96
The R2 for the Random Forest in the TEST  set is 0.95


In [16]:
y_pred_test = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred_test))

0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[17243,   840],
       [   76,   924]])

In [17]:
print("The accuracy in the TRAIN set is: {:.3f}".format(accuracy_score(y_train, y_pred_train)))
print("The accuracy in the TEST  set is: {:.3f}".format(accuracy_score(y_test, y_pred_test)))

The accuracy in the TRAIN set is: 0.962
The accuracy in the TEST  set is: 0.952


In [18]:
print("The precision in the TRAIN set is: {:.3f}".format(precision_score(y_train, y_pred_train)))
print("The precision in the TEST  set is: {:.3f}".format(precision_score(y_test, y_pred_test)))

The precision in the TRAIN set is: 0.956
The precision in the TEST  set is: 0.524


In [19]:
print("The recall in the TRAIN set is: {:.3f}".format(recall_score(y_train, y_pred_train)))
print("The recall in the TEST  set is: {:.3f}".format(recall_score(y_test, y_pred_test)))

The recall in the TRAIN set is: 0.968
The recall in the TEST  set is: 0.924


In [20]:
print("The kappa in the TRAIN set is: {:.3f}".format(cohen_kappa_score(y_train, y_pred_train)))
print("The kappa in the TEST  set is: {:.3f}".format(cohen_kappa_score(y_test, y_pred_test)))

The kappa in the TRAIN set is: 0.924
The kappa in the TEST  set is: 0.645


In [21]:
print("The F1 in the TRAIN set is: {:.3f}".format(f1_score(y_train, y_pred_train)))
print("The F1 in the TEST  set is: {:.3f}".format(f1_score(y_test, y_pred_test)))

The F1 in the TRAIN set is: 0.962
The F1 in the TEST  set is: 0.669


Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the bussiness?

In [22]:
### Overall the model is able to predict the donors in the test set relatively well. 
### Just by upsampling the donor samples in the train dataset we were able to relatively accurately
### predict donors in a very much class imbalaced dataset, which is the test set (compared to some
### examples weve seen before in the class XD). The kappa score also looks quite okay for the test set. 

### If we define True positives as donors and true negatives as not donors, it would be really 
### bad in this case to have too many false negatives, meaning, people that were donors but 
### were then classified as not_donors. That would mean that the company would loose money.

### Sine the default of the positive label is "1" in the scores above, the results of the precision_score 
### are a bit lower for the test set as for the train set. Meaning that the model of course 
### is having a bit more trouble predicting the donors in the test set than in the train set.
### Even though the precision score is decent, this could be improved still.
### In this line, we could also strive to improve the f1 score. 

In [23]:
from sklearn.metrics import make_scorer
scorer = make_scorer(f1_score)

In [24]:
from sklearn.model_selection import GridSearchCV

max_depth_choices= [3,5,10]  
min_samples_split_choices = [10,20] 
min_samples_leaf_choices = [10, 20]

In [25]:
grid = {'max_depth': max_depth_choices,
        'min_samples_split': min_samples_split_choices,
        'min_samples_leaf': min_samples_leaf_choices}

In [26]:
model = RandomForestClassifier(max_samples=0.8)
grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 5, scoring=scorer) 

In [27]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(max_samples=0.8),
             param_grid={'max_depth': [3, 5, 10], 'min_samples_leaf': [10, 20],
                         'min_samples_split': [10, 20]},
             scoring=make_scorer(f1_score))

In [28]:
grid_search.best_params_

{'max_depth': 10, 'min_samples_leaf': 20, 'min_samples_split': 10}

In [29]:
clf = RandomForestClassifier(max_depth=10,
                             min_samples_split=10,
                             min_samples_leaf =20,
                             max_samples=0.8)
clf.fit(X_train, y_train)

print("The R2 for the Random Forest in the TRAIN set is {:.2f}".format(clf.score(X_train, y_train)))
print("The R2 for the Random Forest in the TEST  set is {:.2f}".format(clf.score(X_test, y_test)))


The R2 for the Random Forest in the TRAIN set is 1.00
The R2 for the Random Forest in the TEST  set is 1.00


In [30]:
y_pred_test = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred_test))

0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[18083,     0],
       [   50,   950]])

In [31]:
print("The F1 in the TRAIN set is: {:.3f}".format(f1_score(y_train, y_pred_train)))
print("The F1 in the TEST  set is: {:.3f}".format(f1_score(y_test, y_pred_test)))

The F1 in the TRAIN set is: 1.000
The F1 in the TEST  set is: 0.974


In [ ]:
### The F1 score improved a LOT with this method!!!!!!! 